In [1]:
import os

In [6]:
RESULTS_PATH = ""
SEEDS = None # e.g. range(20)

In [27]:
def get_organised_paths(results_path, seeds):
    timestamps = os.listdir(results_path)
    data_logger_paths = {s: {} for s in seeds}
    
    for timestamp in timestamps:
        timestamp_experiments = os.listdir(os.path.join(results_path, timestamp))
        for exp in timestamp_experiments:
            split_name = exp.split("_")
            seed = int(split_name[1][1:])
            ra = float(split_name[2][2:])
            fa = float(split_name[3][2:])
            data_logger_paths[seed][(ra, fa)] = os.path.join(results_path, timestamp, exp, str(seed), "data_logger.csv")
        
    return data_logger_paths

In [28]:
organised_paths = get_organised_paths(RESULTS_PATH, SEEDS)

FileNotFoundError: [Errno 2] No such file or directory: ''

In [29]:
import tempfile

def test_get_organised_paths():

    with tempfile.TemporaryDirectory() as tmpdirname:
        # create dummmy file structure
        test_dates = ["2021-03-31-21-38-00", "2021-03-31-21-38-02"]
        test_exp_names = {1: "exp_s1_ra0.2_fa0.1", 2: "exp_s2_ra0.2_fa0.8"}
        for test_date in test_dates:
            os.mkdir(os.path.join(tmpdirname, test_date))
            for seed, test_exp in test_exp_names.items():
                os.mkdir(os.path.join(tmpdirname, test_date, test_exp))
                os.mkdir(os.path.join(tmpdirname, test_date, test_exp, str(seed)))
                open(os.path.join(tmpdirname, test_date, test_exp, str(seed), "data_logger.csv"), 'a').close()

        test_data_logger_paths = get_organised_paths(tmpdirname, range(3))

    return test_data_logger_paths

In [30]:
test_get_organised_paths()

{0: {},
 1: {(0.2,
   0.1): '/var/folders/1x/frtldspx7qd6_bnf97dp310w0000gn/T/tmpalj0igx5/2021-03-31-21-38-00/exp_s1_ra0.2_fa0.1/1/data_logger.csv'},
 2: {(0.2,
   0.8): '/var/folders/1x/frtldspx7qd6_bnf97dp310w0000gn/T/tmpalj0igx5/2021-03-31-21-38-00/exp_s2_ra0.2_fa0.8/2/data_logger.csv'}}